In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate
from docx import Document
from reportlab.pdfgen import canvas 
from reportlab.lib.pagesizes import A4
from docx.shared import Inches, Pt, RGBColor
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from pdf2docx import Converter
from PIL import ImageFont
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase import pdfmetrics
from docxcompose.composer import Composer
from reportlab.lib import colors
from io import BytesIO
import locale
from reportlab.lib.styles import ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle
from reportlab.platypus import SimpleDocTemplate, Image, Paragraph, Spacer, Table
from IPython.display import display, Markdown as md
from datetime import datetime
import os
sns.set_style("darkgrid")
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
#data1 = pd.read_csv('relatorios1/cid_ciap_eduardo.csv', sep=';',  encoding='utf-8')
#data2 = pd.read_csv('relatorios1/results2.csv', encoding='utf-8')

In [3]:
data = pd.read_csv('relatorios1/results.csv', encoding='utf-8')

In [ ]:
# new_order_cols = ['cidadao_nome', 'cns', 'nome_mae', 'data_nascimento', 'equipe_vinculo', 'unidade_vinculo']
# data2 = data2[new_order_cols]

In [8]:
data.head()

,tenant,municipio_nome,profissional_categoria,codigo_condicao_saude,nome_condicao_saude,min_data_registro,max_data_registro,total
0,3302858,MESQUITA,enfermeiro,Z029,EXAME NÃO ESPECIFICADO COM FINALIDADES ADMINIS...,2023-03-01,2023-04-30,645
1,3302858,MESQUITA,medico,M472,OUTRAS ESPONDILOSES COM RADICULOPATIAS,2023-03-01,2023-04-30,1
2,3302858,MESQUITA,medico,J03,AMIGDALITE AGUDA,2023-03-01,2023-04-30,148
3,3302858,MESQUITA,enfermeiro,ABP008,DESNUTRIÇÃO,2023-03-01,2023-04-30,1
4,3302858,MESQUITA,medico,B09,INFECÇÃO VIRAL NÃO ESPECIFICADA CARACTERIZADA ...,2023-03-01,2023-04-30,2


In [ ]:
data.info()

In [15]:
# Substitua 'seu_codigo_aqui' pelo código do município fornecido pelo usuário
codigo_municipio = {tenant}

# Filtrar o DataFrame para encontrar a linha onde 'municipio_nome' é igual a 'codigo_municipio'
filtro = (data['tenant'] == codigo_municipio) | (data['municipio_nome'] == codigo_municipio)

# Agora, 'filtro' conterá uma série booleana indicando quais linhas atendem a uma das duas condições
# Se você quiser criar um novo DataFrame apenas com as linhas correspondentes, faça o seguinte:
q_df_filtrado = data[filtro]

# Agora, q_df_filtrado conterá as linhas onde 'tenant' ou 'municipio_nome' é igual a 'codigo_municipio'
q_df_filtrado


NameError: name 'tenant' is not defined

In [ ]:
data.tenant.unique()

In [ ]:
cid_prof = data.pivot_table('total', 'profissional_categoria', aggfunc='sum').reset_index()
soma_total = cid_prof['total'].sum()
cid_prof['porcentagem'] = (cid_prof['total'] / soma_total) * 100

colors = ['dodgerblue', 'darkblue']
plt.figure(figsize=(7, 4))
ax = sns.barplot(x='porcentagem', y='profissional_categoria', data=cid_prof, palette=colors)

for p in ax.patches:
    percentage = f'{p.get_width():.2f}%'  # Formatando o valor para porcentagem com duas casas decimais
    width = p.get_width()
    ax.annotate(percentage, 
                xy=(width, p.get_y() + p.get_height() / 2),
                xytext=(width * 0.01, 0),  # Deslocamento proporcional à largura da barra
                textcoords='offset points',
                va='center', ha='left',
                fontsize=8,
                color='black')

plt.xlabel('Porcentagem')
plt.ylabel('Profissional Categoria')
plt.savefig('graficos/cid_prof.png', bbox_inches='tight')

plt.show()

In [ ]:
profissional_filtrado = 'enfermeiro'
data_filtrado = data[data['profissional_categoria'] == profissional_filtrado]

med_cid = data_filtrado.pivot_table('total', 'nome_condicao_saude', 
                                    aggfunc='sum').sort_values('total', 
                                    ascending=False).head(10).reset_index()

plt.figure(figsize=(7, 4))
ax = sns.barplot(x='total', y='nome_condicao_saude', data=med_cid, color='DodgerBlue')

for p in ax.patches:
    width = p.get_width()
    ax.annotate(f'{int(width):,}', 
                xy=(width, p.get_y() + p.get_height() / 2),
                xytext=(5, 0),  # Deslocamento fixo para as pontas das barras
                textcoords='offset points',
                va='center', ha='left',
                fontsize=8,
                color='black')

plt.xlabel('Total')
plt.ylabel('CID')
plt.savefig('graficos/prof_enf.png', bbox_inches='tight')

plt.show()

In [ ]:
profissional_filtrado = 'medico'
data_filtrado = data[data['profissional_categoria'] == profissional_filtrado]

med_cid = data_filtrado.pivot_table('total', 'nome_condicao_saude', 
                                    aggfunc='sum').sort_values('total', 
                                    ascending=False).head(10).reset_index()

plt.figure(figsize=(7, 4))
ax = sns.barplot(x='total', y='nome_condicao_saude', data=med_cid, color='DarkBlue')

for p in ax.patches:
    width = p.get_width()
    ax.annotate(f'{int(width):,}', 
                xy=(width, p.get_y() + p.get_height() / 2),
                xytext=(5, 0),  # Deslocamento fixo para as pontas das barras
                textcoords='offset points',
                va='center', ha='left',
                fontsize=8,
                color='black')

plt.xlabel('Total')
plt.ylabel('CID')
plt.savefig('graficos/prof_med.png', bbox_inches='tight')

plt.show()

In [ ]:
data['min_data_registro'] = pd.to_datetime(data['min_data_registro'])
data['max_data_registro'] = pd.to_datetime(data['max_data_registro'])

data_minima = data['min_data_registro'].min()
data_maxima = data['max_data_registro'].max()

data_minima_texto = data_minima.strftime("%d/%m/%Y")
data_maxima_texto = data_maxima.strftime("%d/%m/%Y")

print(data_minima_texto)
print(data_maxima_texto)

In [ ]:
locale.setlocale(locale.LC_ALL, '')

quantidade_distinta = data['nome_condicao_saude'].nunique()
quantidade_registros = data['total'].sum()

quantidade_distinta = locale.format_string('%d', quantidade_distinta, grouping=True)
quantidade_registros = locale.format_string('%d', quantidade_registros, grouping=True)

print(quantidade_distinta)
print(quantidade_registros)

In [ ]:
def mp(mm):
        return mm / 0.352777

In [ ]:
def generate_pdf2(title1, logo_image1, graph_image1, graph_image2, description1, description2, output_path2):
    cnv = canvas.Canvas(output_path2, pagesize=A4)

    page_width, page_height = A4
    border_margin = 10  # Margin to avoid content getting cut off

    # Draw a border around the content of the PDF
    cnv.rect(border_margin, border_margin, page_width - 2 * border_margin, page_height - 2 * border_margin)

    font_name = 'Helvetica-Bold'
    font_size = 16
    x = 35
    y = 715
    lines = title1.split('\n')
    cnv.setFont(font_name, font_size)
    for line in lines:
        cnv.drawString(x, y, line)
        y -= 20

    # Logo
    cnv.drawImage(logo_image1, 15, 760, width=120, height=60)

    # Graph 1
    font_name = 'Helvetica-Bold'
    font_size = 13
    x = 35
    y = 685
    y -= 60
    lines = description1.split('\n')
    cnv.setFont(font_name, font_size)
    for line in lines:
        cnv.drawString(x, y, line)
        y -= 20

    cnv.drawImage(graph_image1, 20, 350, width=550, height=220)
    y -= 300

    # Graph 2
    font_name = 'Helvetica-Bold'
    font_size = 13
    x = 35
    y = 345
    y -= 20  # Space between the graphs
    lines = description2.split('\n')
    cnv.setFont(font_name, font_size)
    for line in lines:
        cnv.drawString(x, y, line)
        y -= 20

    cnv.drawImage(graph_image2, 20, 50, width=550, height=220)

    cnv.save()

In [ ]:
profissional_filtrado = 'enfermeiro'
data_filtrado = data[data['profissional_categoria'] == profissional_filtrado]

enf_cid = data_filtrado.pivot_table('total', 'nome_condicao_saude',
                                    aggfunc='sum').sort_values('total',
                                                                ascending=False).head(10).reset_index()

profissional_filtrado = 'medico'
data_filtrado = data[data['profissional_categoria'] == profissional_filtrado]

med_cid = data_filtrado.pivot_table('total', 'nome_condicao_saude',
                                    aggfunc='sum').sort_values('total',
                                                                ascending=False).head(10).reset_index()

title1 = 'As informações apresentadas a seguir são referentes aos 10 códigos \nutilizados, separados por categoria profissional.'

logo_image1 = "imagens/logo.jpeg"
graph_image1 = "graficos/prof_enf.png"
graph_image2 = "graficos/prof_med.png"
output_path2 = "PDFs/prof_enf_med.pdf"

description_enf = (
    f"Grafico 2 - Quantidade de registros de CIDs mais utilizados pelos profissionais \nenfermeiros da APS do municipio {tenant}, no periodo entre {data_minima_texto} a {data_maxima_texto}"
)
description_med = (
    f"Grafico 3 - Quantidade de registros de CIDs mais utilizados pelos profissionais \nmédicos da APS do municipio {tenant}, no periodo entre {data_minima_texto} a {data_maxima_texto}"
)
generate_pdf2(title1, logo_image1, graph_image1, graph_image2, description_enf, description_med, output_path2)

In [ ]:
def generate_pdf_with_table(title, title2, logo_image, graph_image, description, description_quadro, table_data, output_path):
    # Criar o objeto cnv para o PDF
    cnv = canvas.Canvas(output_path, pagesize=A4)

    # Definir as dimensões da página
    page_width, page_height = A4
    border_margin = 1  # Margem para evitar que a borda seja cortada

    # Desenhar uma borda em torno de todo o conteúdo do PDF
    border_x1 = border_margin
    border_y1 = border_margin
    border_x2 = page_width - border_margin
    border_y2 = page_height - border_margin
    cnv.rect(border_x1, border_y1, border_x2 - border_x1, border_y2 - border_y1)

    # Título
    font_name = 'Helvetica-Bold'
    font_size = 16
    x = 200
    y = 730
    lines = title.split('\n')
    cnv.setFont(font_name, font_size)
    for line in lines:
        cnv.drawString(x, y, line)
        y -= 20

    font_name = 'Helvetica'
    font_size = 13
    x = 70
    y = 635
    lines = title2.split('\n')
    cnv.setFont(font_name, font_size)
    for line in lines:
        cnv.drawString(x, y, line)
        y -= 20

    # Inserir imagens
    cnv.drawImage(logo_image, 65, 750, width=120, height=60)
    cnv.drawImage(graph_image, 75, 10, width=400, height=200)

    font_name = 'Helvetica-Bold'
    font_size = 13
    x = 70
    y = 250
    lines = description.split('\n')
    cnv.setFont(font_name, font_size)
    for line in lines:
        cnv.drawString(x, y, line)
        y -= 20

    font_name = 'Helvetica-Bold'
    font_size = 13
    x = 70
    y = 400
    lines = description_quadro.split('\n')
    cnv.setFont(font_name, font_size)
    for line in lines:
        cnv.drawString(x, y, line)
        y -= 20

    # Tabela
    table_width = 450
    table_height = 100
    cell_height = 20
    cell_width = table_width / len(table_data[0])

    font_name = 'Helvetica'
    font_size = 12
    cnv.setFont(font_name, font_size)

    for row_index, row in enumerate(table_data):
        for col_index, cell in enumerate(row):
            x = 70 + col_index * cell_width
            y = 350 - row_index * cell_height

            if row_index == 0:
                cnv.setFillColor('#808080')
                cnv.rect(x, y, cell_width, cell_height, fill=True)
                cnv.setFillColor('black')
                cnv.drawString(x + 5, y + 5, str(cell))
            elif row_index == len(table_data) - 1:
                cnv.setFont(font_name + '-Bold', font_size)
                cnv.setFillColor('#ADD8E6')
                cnv.rect(x, y, cell_width, cell_height, fill=True)
                cnv.setFillColor('black')
                cnv.drawString(x + 5, y + 5, str(cell))
                cnv.setFont(font_name, font_size)
            else:
                cnv.setFillColor('#ADD8E6')
                cnv.rect(x, y, cell_width, cell_height, fill=True)
                cnv.setFillColor('black')
                cnv.drawString(x + 5, y + 5, str(cell))

    # Salvar o PDF
    cnv.save()

In [ ]:
cid_prof = data.pivot_table('total', 'profissional_categoria', aggfunc='sum').reset_index()
soma_total = cid_prof['total'].sum()
cid_prof['porcentagem'] = (cid_prof['total'] / soma_total) * 100

table_data = [["Profissional Categoria", "Total"]]
for index, row in cid_prof.iterrows():
    total_formatado = locale.format_string('%d', row['total'], grouping=True)
    table_data.append([row['profissional_categoria'], total_formatado])

soma_total_formatada = locale.format_string('%d', soma_total, grouping=True)
table_data.append(["Total Geral", soma_total_formatada])

title = f"REGISTRO DE CIDs DA APS \nJUNHO E JULHO DE 2023 \n\n               {tenant}"

title2 = (
        f"       Este relatório analisa os CIDs (Classificação Internacional de Doenças) e \n"
        f"CIAPs (Classificação Internacional de Atenção Primária) mais utilizados na \n"
        f"Atenção Primária à Saúde (APS) municipal e objetiva identificar o perfil de \n"
        f"pacientes atendidos e possíveis padrões de registros dos códigos.\n"
        f"       A equipe de consultoria extraiu os dados, referentes a consultas médicas \n"
        f"e de enfermagem, realizadas entre {data_minima_texto} a {data_maxima_texto}, \n"
        f"período correspondente ao segundo quadrimestre de avaliação do Programa \nPrevine Brasil.\n"
        f"       A análise dos dados de atendimentos feitos nas APS - por médicos e \n"
        f"enfermeiros - no período citado revelou a ocorrência de {quantidade_distinta} CIDs distintos, \n"
        f"em um total de {quantidade_registros} registros."
        )
    
logo_image = "imagens/logo.jpeg"
graph_image = "graficos/cid_prof.png"
output_path = "PDFs/cid_prof.pdf"

description = (
        f"Grafico 1 - Proporção de registros de CIDs por categoria profissional nas \nAPS no municipio {tenant}"
)

description_quadro = (
        f"Qadro 1 - Quantidade de registros de CIDs por categoria profissional nas \nAPS no municipio {tenant}, entre {data_minima_texto} a {data_maxima_texto} "
)

generate_table(cnv, table_data, x_position=70, y_position=290)

generate_pdf_with_table(title, title2, logo_image, graph_image, description, description_quadro, table_data, output_path)

In [ ]:
pdf_files_directory = 'PDFs'
output_pdf_path = 'PDFs/pdf_final_{}_{}.pdf'.format(tenant, datetime.now().strftime('%Y%m%d_%H%M'))

# Nomes dos PDFs que você quer manter nessa ordem
pdf_names_order = ['cid_prof.pdf', 'prof_enf_med.pdf']

# Pega todos os PDFs na pasta PDFs
pdf_files = [file for file in os.listdir(pdf_files_directory) if file.endswith('.pdf')]

# Remove os PDFs que estão na ordem desejada para evitar duplicatas
pdf_files = [file for file in pdf_files if file not in pdf_names_order]

# Cria uma lista final de PDFs com a ordem desejada
final_pdf_files = pdf_names_order + pdf_files

merger = PdfMerger()

# Adiciona todos os PDFs na ordem especificada
for pdf_file in final_pdf_files:
    pdf_path = os.path.join(pdf_files_directory, pdf_file)
    merger.append(pdf_path)


# Salva o PDF final
with open(output_pdf_path, 'wb') as output_pdf:
    merger.write(output_pdf)

print("PDF saved.")